# LCO Colab Procedures

Procedures for executing **Lexicographic Constraint Optimization (LCO)** simulations in Google Colab.

This notebook implements the two-tier static LCO example described in the paper:

- Tier $\mathcal{L}_2$: maximize expected revenue
- Tier $\mathcal{L}_3$: minimize expected overbooking slack subject to a revenue floor


In [ ]:
%%capture
!pip install pyomo highspy
!apt-get install -y coinor-cbc

In [ ]:
from pyomo.environ import (
    ConcreteModel, Set, Var, Param, Binary, NonNegativeReals,
    Constraint, Objective, maximize, minimize, value, SolverFactory
)
import pandas as pd
import numpy as np


## Define synthetic instance and LCO model

In [ ]:
DAYS  = 5
ROOMS = 10
days  = list(range(1, DAYS+1))
rooms = list(range(1, ROOMS+1))

CAP = {d: ROOMS for d in days}

bookings = {
    1:  (1, 2, 120, 0.92),
    2:  (1, 3, 110, 0.85),
    3:  (2, 2, 150, 0.90),
    4:  (2, 3, 130, 0.80),
    5:  (3, 2, 140, 0.88),
    6:  (3, 3, 100, 0.83),
    7:  (4, 2, 160, 0.87),
    8:  (4, 2, 115, 0.78),
    9:  (5, 1, 200, 0.95),
    10: (1, 1, 180, 0.90),
    11: (2, 1, 170, 0.82),
    12: (3, 1, 175, 0.89),
}

B = list(bookings.keys())

def stay_days(bid):
    s, L, _, _ = bookings[bid]
    return [d for d in days if s <= d < s + L]

def build_model():
    m = ConcreteModel()
    m.B = Set(initialize=B)
    m.R = Set(initialize=rooms)
    m.D = Set(initialize=days)

    m.start = Param(m.B, initialize={b: bookings[b][0] for b in B})
    m.len   = Param(m.B, initialize={b: bookings[b][1] for b in B})
    m.price = Param(m.B, initialize={b: bookings[b][2] for b in B})
    m.showp = Param(m.B, initialize={b: bookings[b][3] for b in B})
    m.cap   = Param(m.D, initialize=CAP)

    in_stay = {(b, d) for b in B for d in days if d in stay_days(b)}
    m.InStay = Set(dimen=2, initialize=in_stay)

    yidx = {(b, r, d) for (b, d) in in_stay for r in rooms}
    m.YIDX = Set(dimen=3, initialize=yidx)

    cont = {(b, r, d) for b in B for r in rooms
            for d in days if d in stay_days(b) and (d+1) in stay_days(b)}
    m.ContPair = Set(dimen=3, initialize=cont)

    m.a = Var(m.B, within=Binary)
    m.y = Var(m.YIDX, within=Binary)
    m.w = Var(m.D, within=NonNegativeReals)

    def room_excl(m, r, d):
        return sum(m.y[b, r, d] for b in m.B if (b, d) in m.InStay) <= 1
    m.RoomExcl = Constraint(m.R, m.D, rule=room_excl)

    def assign_if_accepted(m, b, d):
        if (b, d) not in m.InStay:
            return Constraint.Skip
        return sum(m.y[b, r, d] for r in m.R) == m.a[b]
    m.Assign = Constraint(m.B, m.D, rule=assign_if_accepted)

    def continuity(m, b, r, d):
        return m.y[b, r, d] == m.y[b, r, d+1]
    m.Continuity = Constraint(m.ContPair, rule=continuity)

    def overbook_slack(m, d):
        expected = sum(m.a[b] * m.showp[b] for b in m.B if (b, d) in m.InStay)
        return m.w[d] >= expected - m.cap[d]
    m.OverbookSlack = Constraint(m.D, rule=overbook_slack)

    m.RevenueExpr = sum(m.a[b] * m.price[b] * m.len[b] for b in m.B)
    m.obj = Objective(expr=m.RevenueExpr, sense=maximize)
    return m


## Solve Tier $\mathcal{L}_2$ (revenue) and Tier $\mathcal{L}_3$ (overbooking)

In [ ]:
m = build_model()

solver = None
for cand in ["highs", "cbc"]:
    try:
        if SolverFactory(cand).available(exception_flag=False):
            solver = cand
            break
    except Exception:
        pass

if solver is None:
    raise RuntimeError("No MILP solver (highs or cbc) is available in this environment.")

opt = SolverFactory(solver)
print(f"Using solver: {solver}")

res2 = opt.solve(m, tee=False)
Z2 = value(m.RevenueExpr)
print("Tier L2 (Revenue) optimal value Z2* =", Z2)

eps = 1e-6
m.RevenueFloor = Constraint(expr=m.RevenueExpr >= Z2 - eps)
m.del_component(m.obj)
m.obj = Objective(expr=sum(m.w[d] for d in m.D), sense=minimize)

res3 = opt.solve(m, tee=False)
Z3 = sum(value(m.w[d]) for d in m.D)
print("Tier L3 (Overbooking slack) optimal sum =", Z3)


## Extract KPIs and assignment summary

In [ ]:
assignments = []
for b in m.B:
    if value(m.a[b]) > 0.5:
        sdays = stay_days(b)
        assigned_r = None
        for r in m.R:
            ok = True
            for d in sdays:
                if (b, r, d) not in m.YIDX or value(m.y[b, r, d]) <= 0.5:
                    ok = False
                    break
            if ok:
                assigned_r = r
                break
        assignments.append((b, sdays, assigned_r))

print("Accepted bookings and assigned room:")
for b, days_b, r in assignments:
    print(f"Booking {b}: stay days {days_b}, room {r}")

print("\nDaily overbooking slack w_d:")
for d in m.D:
    print(f"Day {d}: w_d = {value(m.w[d]):.4f}")
